In [508]:
#gates in python with next-state logic, fml this was so obvious
from functools import reduce
from operator import iand
from operator import ior

In [509]:
%%bash --out input
cat mux_example.txt

In [510]:
#format it
input=input.splitlines()

In [511]:
num_inputs=int(input[0])
num_outputs=int(input[1])
input_names=input[2:3+num_inputs-1]
input_names=[x.strip() for x in input_names]
print(num_inputs)
print(num_outputs)
print(input_names)

3
1
['I0', 'A', 'I1']


In [512]:
num_gates=int(input[2+num_inputs])
print(num_gates)

4


In [513]:
gate_names=input[3+num_inputs:3+num_gates+num_inputs]
gate_names=[x.strip() for x in gate_names]
input_locations=input[num_gates+1+num_inputs+2:num_gates+1+num_inputs+2+num_inputs]
print(input_locations)

['NAND0 ', 'NAND0 NAND1 NAND1', 'NAND3']


In [514]:
gate_inputs=input[num_gates+1+num_inputs+2+num_inputs:num_gates+1+num_inputs+2+num_inputs+num_gates]
gate_inputs=[x.strip() for x in gate_inputs]
print(gate_inputs)

['', '', 'NAND0 NAND3', 'NAND1']


In [515]:
gate_types=input[num_gates+1+num_inputs+2+num_inputs+num_gates:num_gates+1+num_inputs+2+num_inputs+num_gates*2]
output_gate=input[num_gates+1+num_inputs+2+num_inputs+num_gates*2]
output_gate=output_gate.strip()
num_cycles=int(input[num_gates+1+num_inputs+2+num_inputs+num_gates*2+1])
verbose=int(input[num_gates+1+num_inputs+2+num_inputs+num_gates*2+2])==1
print(gate_types)
print(output_gate)
print(num_cycles)
print(verbose)

['2', '2', '2', '2']
NAND2
10
False


In [520]:
all_perms=[]
def permute(sz, str=""):
    if len(str)==sz:
        all_perms.append(str)
        return
    for bit in '01':
        permute(sz,str+bit)
permute(len(input_names))
print(all_perms)
truth_table_output=[]
oLoc=gate_names.index(output_gate)

['000', '001', '010', '011', '100', '101', '110', '111']


In [521]:
for curr_input_perm in all_perms:
    OUTPUT=[[] for i in range(num_cycles)]
    #input_arr=[1 for i in range(num_gates)]
    #TODO REMOVE HARDCODING
    input_arr=list(map(int,list(curr_input_perm)))
    current_state=[0 for i in range(num_gates)]
    next_state=[0 for i in range(num_gates)]
    gate_deps=[[] for i in range(num_gates)]
    counter=0
    for i in input_locations:
        temp_arr=i.split()
        for i in temp_arr:
            loc=gate_names.index(i)
            gate_deps[loc].append(-(counter+1))
        counter=counter+1
    for i in range(num_gates):
        temp_arr=gate_inputs[i].split()
        for j in temp_arr:
            gate_deps[i].append(gate_names.index(j))
    #gate_deps = [list(set(x)) for x in gate_deps]
    def logic(dependencies,gate_type):
        curr_arr=[]
        for i in dependencies:
            if (i<0):
                curr_arr.append(input_arr[-1*i -1])
            else:
                curr_arr.append(current_state[i])
        if gate_type == 0:
            return reduce(lambda x, y: x & y, curr_arr)
        elif gate_type == 1:
            return reduce(lambda x, y: x | y, curr_arr)
        elif gate_type == 2:
            return (reduce(lambda x, y: not (x & y), curr_arr))
    for i in range(num_cycles):
        OUTPUT[i]=current_state.copy()
        for j in range(num_gates):
            next_state[j]=logic(gate_deps[j],int(gate_types[j]))
            #print(next_state)
        current_state=next_state
    def remove_adjacent(nums):
         return [a for a,b in zip(nums, nums[1:]+[not nums[-1]]) if a != b]
    if not verbose:
        OUTPUT=remove_adjacent(OUTPUT)
    truth_table_output.append(OUTPUT[-1][oLoc])

In [532]:
str=""
for i in input_names:
    str+='{:16} '.format(i)
str+='\t\t{:16} '.format(gate_names[oLoc])
print(str)
str=""
count=0
for i in all_perms:
    for j in i:
        str+='{:16} '.format(j)
    str+='{:16} '.format(truth_table_output[count])
    count+=1
    print(str)
    str=""

I0               A                I1               		NAND2            
0                0                0                               0 
0                0                1                               1 
0                1                0                               0 
0                1                1                               0 
1                0                0                               0 
1                0                1                               1 
1                1                0                               1 
1                1                1                               1 


In [1]:
!ls

BigHeader.h		  a.cpp		 cpu.h	 mutex.h	   semaphore.h
Logic_Gates_Helper.ipynb  blank_img.png  cv.h	 mux_example.txt   thread.h
Makefile		  core		 disk.h  sample_input.txt


In [2]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig = plt.figure(figsize=(20,30))

img=mpimg.imread('blank_img.png')

def onclick(event):
    ix, iy = event.xdata, event.ydata
    print(ix, iy)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

imgplot = plt.imshow(img)
plt.show()

TclError: couldn't connect to display ":0"

In [ ]:
# instr='{:16}{:30}{:50}'.format("Gates\t|","Gate Dependencies\t\t\t|","Cycles")
# print(instr)
# for i in gate_names:
#     cLoc=gate_names.index(i)
#     instr='{:16}'.format("Gate "+i+"\t|")
#     print(instr,end=" ")
#     str=[]
#     for x in gate_deps[cLoc]:
#         if x<0:
#             str.append(input_names[-1*x-1])
#         else:
#             str.append(gate_names[x])
#     if len(str)>1:
#         if int(gate_types[cLoc]) == 0:
#             for b in range(0,len(str)):
#                 str.insert(b*2,'&')
#         elif int(gate_types[cLoc]) == 1:
#             for b in range(0,len(str)):
#                 str.insert(b*2,'|')
#         elif int(gate_types[cLoc]) == 2:
#             for b in range(0,len(str)):
#                 str.insert(b*2,'!&')
#         str.pop(0)
#     str=''.join(str)
#     str='{:30}\t|'.format(str)
#     print(str,end=" ")
#     for j in OUTPUT:
#         nstr='{:1} |'.format(j[cLoc])
#         print(nstr,end=" ")
#     print()
    
# print("------------------------------------------------------------------------------")
# print("OUTPUT OVER TIME: ",end=" ")
# for j in OUTPUT:
#     print(j[gate_names.index(output_gate)],end=" ")
#     if not (j == OUTPUT[-1]):
#         print(" -> ", end=" ")